In [32]:
import numpy as np 
import scipy.io
import scipy.stats
import math

#### Auxiliary functions

In [33]:
def log_normpdf(x, mu, sigma):
    """
      Computes the natural logarithm of the normal probability density function

    """
    prob = scipy.stats.norm(mu, sigma).pdf(x)
    return prob


In [34]:
class JointParts():
    def __init__(self, N_class):
        self.means = np.zeros([3,N_class])
        self.sigma = np.zeros([3,N_class])


In [35]:
class Model():
    def __init__(self, N_class, N_joints):
        self.jointparts = [JointParts(N_class) for i in range(N_joints)]
        self.class_priors = {}
        

In [36]:
def cal_acc(a, b):
    """
       Input
           two vectors with same size

       Output
           accuracy
    """

    acc = 0.0
    for i in range(len(a)):
        if(a[i] == b[i]):
            acc += 1
    acc = acc / len(a)

    return acc

#### Functions to implement

In [37]:
def fit_model(X):
    """
      Compute the mean and variance of X, 
    """
    
    # TODO
    mean = 0.0
    for i in range(len(X)):
        mean += X[i]
    mean = mean / len(X)
    
    variance = 0.0
    for i in range(len(X)):
        tmp = X[i] - mean
        variance += tmp * tmp.T
    variance = variance / len(X)
    

    return (mean, variance)


In [38]:
def learn_model(dataset, labels, G=None):
    """
    Input:
     dataset: The data as loaded 
     labels:  The labels as loaded 
     
    Output: the model
     a (tentative) structure for the output model is:
       model.class_priors: containing a vector with the prior estimations
                           for each class
       model.jointparts[i] contains the estimated parameters for the i-th joint

            model.jointparts(i).means: a matrix of 3 x #classes with the
                   estimated means for each of the x,y,z variables of the 
                   i-th joint and for each class.
            model.jointparts(i).sigma: a matrix of 3 x #classes with the
                   estimated stadar deviations for each of the x,y,z 
                   variables of the i-th joint and for each class.
    """

    # TODO
    ##1st Step: obtain N_samples,N_joints,N_class
    N_samples = labels.shape[0]   #get training_data total samples
    N_joints = dataset.shape[0]   #get training_data total joints
    #get the total classes using bincount&count_nonzero to count
    counts = np.bincount(labels.reshape(-1))   
    N_class = np.count_nonzero(counts)

    ##2nd Step: load Model class
    model = Model(N_class=N_class, N_joints=N_joints)
    
    ##3.1 Step: compute the priori estimations
    for i in range(len(counts)):
        if counts[i] > 0:
            model.class_priors[int(i)] = counts[i] / N_samples  #calculate the priori estimations

    ##3.2 Step: obtain each different classes' index
    classIndex = []
    for i in range(len(counts)):
        if counts[i] > 0:
            nums = np.where(labels == i)
            classIndex.append(nums[0])  #obtain the index of different classes
    
    ##3.3 Step: obtain means and standard deviations for each of the x,y,z variables of the i-th joint and for each class
    for i in range(N_joints):
        #3 * N_class model.jointparts[i].means
        for j in range(N_class):
            for a in range(3):
                vec = dataset[i, a, classIndex[j]] #construct joint vector from each classes index
                mean, var = fit_model(vec)
                model.jointparts[i].means[a][j] = mean
                model.jointparts[i].sigma[a][j] = math.sqrt(var)

    return model

In [39]:
def classify_samples(instances, model):
    """    
    Input
       instance: a 20x3x#instances matrix defining body positions of
                 instances
       model: as the output of learn_model

    Output
       probs: a matrix of #instances x #classes with the probability of each
              instance of belonging to each of the classes

    Important: to avoid underflow numerical issues this computations should
               be performed in log space
    """
    
    # TODO
    ##1st Step: obtain N_samples,N_joints,N_class
    N_samples = instances.shape[-1]
    N_class = len(model.class_priors)
    N_joints = instances.shape[0]
    
    ##construct a dict for different classes
    indexDict = {}  #index the relevant key
    index = 0
    for k in model.class_priors.keys():
        indexDict[index] = k
        index += 1

    ##construct probs non-matrix for N_samples*N_class
    probs = np.zeros([N_samples, N_class])
    
    #compute the probability for each instance of each classes
    for i in range(N_samples):
        data = instances[:, :, i]
        for k in range(N_class):
            prob = 1
            for j in range(N_joints):
                for a in range(3):
                    mean = model.jointparts[j].means[a][k]   # Obtain eachinstance of each classes mean and standar var
                    sigma = model.jointparts[j].sigma[a][k]
                    prob *= log_normpdf(data[j, a], mean, sigma)  # Computes the natural logarithm of the normal PDF
            index = indexDict[k]
            prob *= model.class_priors[index] #accumulate each class joint priors and likelihood to obtain prob
            probs[i][k] = prob
            
#     probs = likelihood * priors

    return probs


# TEST PART

## 1 learn model

In [40]:
dd = scipy.io.loadmat('data/validation_data.mat')
X = dd['data_small']
Y = dd['labels_small']
# the index for training set
ind = dd['train_indexes']
[trainInd,nouse] = np.where(ind==1)
# get the training set with fixed index
X_train = X[:,:,trainInd]
y_train = Y[trainInd]
# training/learning
modelNB = learn_model(X_train, y_train)

## 2 Classify

In [41]:
dd = scipy.io.loadmat('data/validation_data.mat')
X = dd['data_small']
Y = dd['labels_small']

# get the test set index
ind = dd['test_indexes']
[testInd,nouse] = np.where(ind==1)
# get the testing set with fixed index
X_test = X[:,:,testInd]
y_test = Y[testInd]
# classify  
y_rst = classify_samples(X_test, modelNB)
labelpos = np.argmax(y_rst, axis = 1)
# get the label of the each instance with probs
classPriors = modelNB.class_priors
whatlabel = list(classPriors.keys())
y = classify_samples(X_test, modelNB)
labelpos = np.argmax(y, axis = 1)

rst = [whatlabel[labelpos[i]] for i in range(len(labelpos))]
val = list(y_test.reshape(len(y_test),))

print(cal_acc(rst,val))

0.95
